## Overall Workflow

1. Initialize regressors 
2. Build the pipelines
3. Set up the parameter grids
4. Set up multiple GridSearchCV for the hyperparameter tuning process of each algorithm. This takes place in the inner loop.
5. Define the inner and outer loops
* Use StratifiedKFold to create outer loop folds
* Training folds created in the outerloop will be used in the inner loop for parameter tuning.
* The inner loop selects the best hyperparameter setting. The best hyperparameter will be evaluated on both the average across inner test folds and the *one* corresponding test fold of the outer loop.

In [78]:
%matplotlib notebook
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

#Set random seed to current date and time to get a pseudorandom state
import random
from datetime import datetime
random.seed(datetime.now()) 

#This is to round the output to 3 decimal places when printing outputs
np.set_printoptions(precision=3)


In [99]:

#Import dataset
data = pd.read_csv("C:/Users/kimng/Desktop/ML - Age, Hipp, CVLT/CVLTHippocampus.csv")
data = data.reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Columns: 206 entries, ID_code to RecallEfficiency
dtypes: float64(181), int64(22), object(3)
memory usage: 225.4+ KB


In [100]:
data.columns

Index(['ID_code', 'Scan_code', 'Sex', 'Age', 'Age50', 'AgeCategory', 'ICV',
       'Handedness', 'Education', 'EduYears',
       ...
       'CVLT_CorrectRate_B', 'CVLT_CorrectRate_SDFR', 'CVLT_CorrectRate_SDCR',
       'CVLT_CorrectRate_LDFR', 'CVLT_CorrectRate_LDCR', 'GenVerbLearn',
       'ResponseError', 'OrgStrat', 'SerialEffect', 'RecallEfficiency'],
      dtype='object', length=206)

In [101]:
columns = ['CVLT_Imm_Total', 'CVLT_DelR_SD_Free', 'CVLT_DelR_LD_Free',
            'Age','Sex', 'EduYears', 'Smoker', 'High_BP', 'COMT', 'BDNF2', 'ApoE4',
           'L_HH_Total', 'R_HH_Total', 'L_HB_Total', 'R_HB_Total', 'L_HT_Total', 'R_HT_Total',
           'L_DG_Total', 'R_DG_Total',
           'L_CA_Total', 'R_CA_Total',
           'L_Sub_Total', 'R_Sub_Total',
           'L_HH_CA', 'R_HH_CA', 'L_HB_CA', 'R_HB_CA', 'L_HT_CA', 'R_HT_CA', 
           'L_HH_DG', 'R_HH_DG', 'L_HB_DG', 'R_HB_DG', 'L_HT_DG', 'R_HT_DG',
           'L_HH_Sub', 'R_HH_Sub', 'L_HB_Sub', 'R_HB_Sub', 'L_HT_Sub', 'R_HT_Sub']


In [102]:
df = data[columns]

In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CVLT_Imm_Total     137 non-null    float64
 1   CVLT_DelR_SD_Free  137 non-null    float64
 2   CVLT_DelR_LD_Free  137 non-null    float64
 3   Age                140 non-null    int64  
 4   Sex                140 non-null    int64  
 5   EduYears           140 non-null    int64  
 6   Smoker             140 non-null    int64  
 7   High_BP            140 non-null    int64  
 8   COMT               140 non-null    int64  
 9   BDNF2              140 non-null    int64  
 10  ApoE4              140 non-null    int64  
 11  L_HH_Total         129 non-null    float64
 12  R_HH_Total         129 non-null    float64
 13  L_HB_Total         129 non-null    float64
 14  R_HB_Total         129 non-null    float64
 15  L_HT_Total         129 non-null    float64
 16  R_HT_Total         129 non

In [104]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129 entries, 0 to 139
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CVLT_Imm_Total     129 non-null    float64
 1   CVLT_DelR_SD_Free  129 non-null    float64
 2   CVLT_DelR_LD_Free  129 non-null    float64
 3   Age                129 non-null    int64  
 4   Sex                129 non-null    int64  
 5   EduYears           129 non-null    int64  
 6   Smoker             129 non-null    int64  
 7   High_BP            129 non-null    int64  
 8   COMT               129 non-null    int64  
 9   BDNF2              129 non-null    int64  
 10  ApoE4              129 non-null    int64  
 11  L_HH_Total         129 non-null    float64
 12  R_HH_Total         129 non-null    float64
 13  L_HB_Total         129 non-null    float64
 14  R_HB_Total         129 non-null    float64
 15  L_HT_Total         129 non-null    float64
 16  R_HT_Total         129 non

In [105]:
#df.Sex = df.Sex-1

In [127]:
df = df.reset_index()

In [108]:
df.loc[df.isnull().any(axis=1)]

,index,CVLT_Imm_Total,CVLT_DelR_SD_Free,CVLT_DelR_LD_Free,Age,Sex,EduYears,Smoker,High_BP,COMT,...,L_HB_DG,R_HB_DG,L_HT_DG,R_HT_DG,L_HH_Sub,R_HH_Sub,L_HB_Sub,R_HB_Sub,L_HT_Sub,R_HT_Sub


### Create category variable for stratification

In [109]:
#Bin Age into groups
bins = [10, 20, 30, 40, 50, 60, 70, 80, 90]
labels = [1,2,3,4,5,6,7,8]
df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels)
df = df.reset_index(drop=True)

#Function for categorise dataframe
def categorise(row):
    if row['AgeGroup'] == 1 and row['Sex'] == 0:
        return 1
    elif row['AgeGroup'] == 1 and row['Sex'] == 1:
        return 2
    elif row['AgeGroup'] == 2 and row['Sex'] == 0:
        return 3
    elif row['AgeGroup'] == 2 and row['Sex'] == 1:
        return 4
    elif row['AgeGroup'] == 3 and row['Sex'] == 0:
        return 5
    elif row['AgeGroup'] == 3 and row['Sex'] == 1:
        return 6
    elif row['AgeGroup'] == 4 and row['Sex'] == 0:
        return 7
    elif row['AgeGroup'] == 4 and row['Sex'] == 1:
        return 8
    elif row['AgeGroup'] == 5 and row['Sex'] == 0:
        return 9
    elif row['AgeGroup'] == 5 and row['Sex'] == 1:
        return 10
    elif row['AgeGroup'] == 6 and row['Sex'] == 0:
        return 11
    elif row['AgeGroup'] == 6 and row['Sex'] == 1:
        return 12
    elif row['AgeGroup'] == 7 and row['Sex'] == 0:
        return 13
    elif row['AgeGroup'] == 7 and row['Sex'] == 1:
        return 14
    elif row['AgeGroup'] == 8 and row['Sex'] == 0:
        return 15
    elif row['AgeGroup'] == 8 and row['Sex'] == 1:
        return 16


#Apply categories to dataframe
df['grp'] = df.apply(lambda row: categorise(row), axis=1)

In [128]:
df.head()

,level_0,index,CVLT_Imm_Total,CVLT_DelR_SD_Free,CVLT_DelR_LD_Free,Age,Sex,EduYears,Smoker,High_BP,...,L_HT_DG,R_HT_DG,L_HH_Sub,R_HH_Sub,L_HB_Sub,R_HB_Sub,L_HT_Sub,R_HT_Sub,AgeGroup,grp
0,0,0,63.0,15.0,16.0,0.194030,1.0,0.461538,1,1,...,0.797652,0.714589,0.480730,0.441510,0.442231,0.787128,0.649385,0.583430,NaN,6
1,1,1,52.0,12.0,12.0,0.477612,0.0,0.615385,1,1,...,0.482203,0.381629,0.485257,0.514386,0.357076,0.548778,0.185754,0.397777,NaN,7
2,2,2,60.0,15.0,15.0,0.179104,1.0,0.461538,1,1,...,0.786276,0.584677,0.626998,0.579404,0.535864,0.963885,0.913314,0.568308,NaN,4
3,3,3,54.0,12.0,11.0,0.731343,1.0,0.230769,1,1,...,0.310528,0.260791,0.353162,0.604380,0.574611,0.548574,0.342245,0.364031,NaN,12
4,4,4,67.0,15.0,14.0,0.014925,1.0,0.307692,1,1,...,0.620906,0.648545,0.285197,0.410578,0.549605,0.336738,0.518550,0.359446,NaN,2


### Initialize Regressors

In [111]:
reg1 = Ridge()
reg2 = Lasso()
reg3 = ElasticNet()
reg4 = KNeighborsRegressor()
reg5 = SVR()
reg6 = DecisionTreeRegressor()
reg7 = RandomForestRegressor()

### Build the pipelines
for Ridge, Lasso, ElasticNet, K-Neighbors, & SVM because feature scaling is required for these algorithms.

In [123]:
pipe1 = Pipeline([
                 #('scaler', MinMaxScaler()),
                  ('reg1', reg1)])
pipe2 = Pipeline([
                 #('scaler', MinMaxScaler()),
                  ('reg2', reg2)])
pipe3 = Pipeline([
                 #('scaler', MinMaxScaler()),
                  ('reg3', reg3)])
pipe4 = Pipeline([
                 #('scaler', MinMaxScaler()),
                  ('reg4', reg4)])
pipe5 = Pipeline([
                 #('scaler', MinMaxScaler()),
                  ('reg5', reg5)])
pipe6 = Pipeline([
                 #('scaler', MinMaxScaler()),
                  ('reg6', reg1)])
pipe7 = Pipeline([
                 #('scaler', MinMaxScaler()),
                  ('reg7', reg2)])


### Setting up the parameter grids

In [119]:
param_grid1 = [{'reg1__alpha': [.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 15, 20]}]

param_grid2 = [{'reg2__alpha': [.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 15, 20]}]

param_grid3 = [{'reg3__alpha': [.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 15, 20]}]

param_grid4 = [{'reg4__n_neighbors': list(range(1, 10))}]

param_grid5 = [{'reg5__C': [0.01, 0.1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40, 50], 
                'reg5__gamma': ['scale', 'auto'], 
                'reg5__kernel': ['rbf', 'linear', 'poly'], 
                'reg5__degree': [1,2,3,4,5,6]}]

param_grid6 = [{'reg6__criterion': ['absolute_error'],
                'reg6__max_depth': [1, 2, 3, 4, 5],
                'reg6__min_samples_leaf': [1, 2, 3, 4, 5]
               }]

param_grid7 = [{'reg7__criterion': ['absolute_error'],
                'reg7__max_depth': [1, 2, 3, 4, 5],
                'reg7__min_samples_leaf': [1, 2, 3, 4, 5],
                'reg7__n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}]

### Setting up Grid Search for inner loops

In [135]:

gridcvs = {}

inner_cv = StratifiedKFold(n_splits=3, shuffle=True)

for pgrid, est, name in zip((param_grid1, param_grid2, param_grid3, param_grid4, param_grid5, param_grid6, param_grid7),
                            (pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe7),
                            ('Ridge', 'Lasso', 'ElasticNet', 'KNN', 'SVR', 'DTree', 'RForest')):
    gcv = GridSearchCV(estimator=est,
                       param_grid=pgrid,
                       scoring='neg_mean_squared_error',
                       n_jobs=-1,
                       cv=inner_cv,                       
                       verbose=0,
                       refit=True
                      )
    gridcvs[name] = gcv

In [121]:
gridcvs

{'Ridge': GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1, shuffle=True),
              estimator=Pipeline(steps=[('reg1', Ridge())]), n_jobs=-1,
              param_grid=[{'reg1__alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5,
                                           1, 5, 10, 15, 20]}],
              scoring='neg_mean_squared_error'),
 'Lasso': GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1, shuffle=True),
              estimator=Pipeline(steps=[('reg2', Lasso())]), n_jobs=-1,
              param_grid=[{'reg2__alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5,
                                           1, 5, 10, 15, 20]}],
              scoring='neg_mean_squared_error'),
 'ElasticNet': GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1, shuffle=True),
              estimator=Pipeline(steps=[('reg3', ElasticNet())]), n_jobs=-1,
              param_grid=[{'reg3__alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5,
                                           1, 5, 10, 

### Nested CV

In [133]:
import warnings
warnings.filterwarnings("ignore")

bins = [10, 20, 30, 40, 50, 60, 70, 80, 90]
labels = [1,2,3,4,5,6,7,8]
df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels)

stratifyCategory = ['Sex', 'AgeGroup']

#from sklearn.model_selection import StratifiedShuffleSplit
#split = StratifiedShuffleSplit(test_size = 24, random_state=42, n_splits = 5)


feature_names = ['Age','Sex', 'EduYears', 'Smoker', 'High_BP', 'COMT', 'BDNF2', 'ApoE4',
                 'L_HH_Total', 'R_HH_Total', 'L_HB_Total', 'R_HB_Total', 'L_HT_Total', 'R_HT_Total',
                 'L_DG_Total', 'R_DG_Total',
                 'L_CA_Total', 'R_CA_Total',
                 'L_Sub_Total', 'R_Sub_Total',
                 'L_HH_CA', 'R_HH_CA', 'L_HB_CA', 'R_HB_CA', 'L_HT_CA', 'R_HT_CA', 
                 'L_HH_DG', 'R_HH_DG', 'L_HB_DG', 'R_HB_DG', 'L_HT_DG', 'R_HT_DG',
                 'L_HH_Sub', 'R_HH_Sub', 'L_HB_Sub', 'R_HB_Sub', 'L_HT_Sub', 'R_HT_Sub']


# Feature Scaling
scaler = MinMaxScaler()
df[feature_names] = scaler.fit_transform(df[feature_names])

for name, est in gridcvs.items():

        print(50 * '-', '\n')
        print('Algorithm:', name)
        print('    Inner loop:')

        outer_scores_mae = []
        outer_scores_mse = []
        outer_scores_r2 = []
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True) 

        for train_index_outer, val_index_outer in outer_cv.split(df, df.grp):
            train_set = df.loc[train_index_outer,:]
            val_set = df.loc[val_index_outer,:]

            X = ['Age','Sex', 'EduYears', 
                     'L_HH_Total', 'R_HH_Total', 'L_HB_Total', 'R_HB_Total', 'L_HT_Total', 'R_HT_Total',
                     'L_DG_Total', 'R_DG_Total',
                     'L_CA_Total', 'R_CA_Total',
                     'L_Sub_Total', 'R_Sub_Total',
                     'L_HH_CA', 'R_HH_CA', 'L_HB_CA', 'R_HB_CA', 'L_HT_CA', 'R_HT_CA', 
                     'L_HH_DG', 'R_HH_DG', 'L_HB_DG', 'R_HB_DG', 'L_HT_DG', 'R_HT_DG',
                     'L_HH_Sub', 'R_HH_Sub', 'L_HB_Sub', 'R_HB_Sub', 'L_HT_Sub', 'R_HT_Sub']
            y = ['CVLT_Imm_Total']

            X_train = train_set[X]
            y_train = train_set[y]
            X_val = val_set[X]
            y_val = val_set[y]
            
            #Apply grid search with CV=3 on outer train_set

            grid = gridcvs[name]
            grid.fit(X = X_train,
                     y = y_train) # run inner loop hyperparam tuning
            print('\n        Best MSE (inner test folds):', (grid.best_score_))
            print('        Best parameters:', grid.best_params_)

            # Calculate evaluation metrics using best-tuned model on out val_set
            #MSE
            y_val_hat = grid.best_estimator_.predict(X_val)
            mse_val = mean_squared_error(y_val, y_val_hat)
            outer_scores_mse.append(mse_val)

            print('        MSE (on outer validation fold)', (outer_scores_mse[-1]))

            # MAE
            mae_val = mean_absolute_error(y_val, y_val_hat)
            outer_scores_mae.append(mae_val)
            
            print('        MAE (on outer validation fold)', (outer_scores_mae[-1]))
            
            # R2
            r2_val = r2_score(y_val, y_val_hat)
            outer_scores_r2.append(r2_val)
            
            print('        R2 (on outer validation fold)', (outer_scores_r2[-1]))
            
        print('\n    Outer Loop:')
        print('        MSE %.2f +/- %.2f'%
                  (np.mean(outer_scores_mse), np.std(outer_scores_mse)))
        print('        MAE %.2f +/- %.2f'%
                  (np.mean(outer_scores_mae), np.std(outer_scores_mae)))
        print('        R2 %.2f +/- %.2f'%
                  (np.mean(outer_scores_r2), np.std(outer_scores_r2)))

-------------------------------------------------- 

Algorithm: Ridge
    Inner loop:

        Best MSE (inner test folds): -63.21832624910068
        Best parameters: {'reg1__alpha': 5}
        MSE (on outer test fold) 67.39387915473645
        MAE (on outer test fold) 7.330299414890193
        R2 (on outer test fold) 0.22736772138384065

        Best MSE (inner test folds): -64.1364187466037
        Best parameters: {'reg1__alpha': 5}
        MSE (on outer test fold) 65.71905764250954
        MAE (on outer test fold) 6.882532033147509
        R2 (on outer test fold) 0.29638766287081963

        Best MSE (inner test folds): -61.863866866826584
        Best parameters: {'reg1__alpha': 1}
        MSE (on outer test fold) 66.33144478557809
        MAE (on outer test fold) 6.524943986794214
        R2 (on outer test fold) 0.07948643711917414

        Best MSE (inner test folds): -59.286150788302564
        Best parameters: {'reg1__alpha': 5}
        MSE (on outer test fold) 69.86245200652

ValueError: Invalid parameter criterion for estimator Ridge(). Check the list of available parameters with `estimator.get_params().keys()`.

In [134]:
import warnings
warnings.filterwarnings("ignore")

bins = [10, 20, 30, 40, 50, 60, 70, 80, 90]
labels = [1,2,3,4,5,6,7,8]
df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels)

stratifyCategory = ['Sex', 'AgeGroup']

#from sklearn.model_selection import StratifiedShuffleSplit
#split = StratifiedShuffleSplit(test_size = 24, random_state=42, n_splits = 5)


feature_names = ['Age','Sex', 'EduYears', 'Smoker', 'High_BP', 'COMT', 'BDNF2', 'ApoE4',
                 'L_HH_Total', 'R_HH_Total', 'L_HB_Total', 'R_HB_Total', 'L_HT_Total', 'R_HT_Total',
                 'L_DG_Total', 'R_DG_Total',
                 'L_CA_Total', 'R_CA_Total',
                 'L_Sub_Total', 'R_Sub_Total',
                 'L_HH_CA', 'R_HH_CA', 'L_HB_CA', 'R_HB_CA', 'L_HT_CA', 'R_HT_CA', 
                 'L_HH_DG', 'R_HH_DG', 'L_HB_DG', 'R_HB_DG', 'L_HT_DG', 'R_HT_DG',
                 'L_HH_Sub', 'R_HH_Sub', 'L_HB_Sub', 'R_HB_Sub', 'L_HT_Sub', 'R_HT_Sub']


# Feature Scaling
scaler = MinMaxScaler()
df[feature_names] = scaler.fit_transform(df[feature_names])

for name, est in gridcvs.items():

        print(50 * '-', '\n')
        print('Algorithm:', name)
        print('    Inner loop:')

        outer_scores_mae = []
        outer_scores_mse = []
        outer_scores_r2 = []
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True) 

        for train_index_outer, val_index_outer in outer_cv.split(df, df.grp):
            train_set = df.loc[train_index_outer,:]
            val_set = df.loc[val_index_outer,:]

            X = ['Age','Sex', 'EduYears', ''
                     'L_HH_Total', 'R_HH_Total', 'L_HB_Total', 'R_HB_Total', 'L_HT_Total', 'R_HT_Total',
                     'L_DG_Total', 'R_DG_Total',
                     'L_CA_Total', 'R_CA_Total',
                     'L_Sub_Total', 'R_Sub_Total',
                     'L_HH_CA', 'R_HH_CA', 'L_HB_CA', 'R_HB_CA', 'L_HT_CA', 'R_HT_CA', 
                     'L_HH_DG', 'R_HH_DG', 'L_HB_DG', 'R_HB_DG', 'L_HT_DG', 'R_HT_DG',
                     'L_HH_Sub', 'R_HH_Sub', 'L_HB_Sub', 'R_HB_Sub', 'L_HT_Sub', 'R_HT_Sub']
            y = ['CVLT_DelR_LD_Free']

            X_train = train_set[X]
            y_train = train_set[y]
            X_val = val_set[X]
            y_val = val_set[y]
            
            #Apply grid search with CV=3 on outer train_set

            grid = gridcvs[name]
            grid.fit(X = X_train,
                     y = y_train) # run inner loop hyperparam tuning
            print('\n        Best MSE (inner test folds):', (grid.best_score_))
            print('        Best parameters:', grid.best_params_)

            # Calculate evaluation metrics using best-tuned model on out val_set
            #MSE
            y_val_hat = grid.best_estimator_.predict(X_val)
            mse_val = mean_squared_error(y_val, y_val_hat)
            outer_scores_mse.append(mse_val)

            print('        MSE (on outer validation fold)', (outer_scores_mse[-1]))

            # MAE
            mae_val = mean_absolute_error(y_val, y_val_hat)
            outer_scores_mae.append(mae_val)
            
            print('        MAE (on outer validation fold)', (outer_scores_mae[-1]))
            
            # R2
            r2_val = r2_score(y_val, y_val_hat)
            outer_scores_r2.append(r2_val)
            
            print('        R2 (on outer validation fold)', (outer_scores_r2[-1]))
            
        print('\n    Outer Loop:')
        print('        MSE %.2f +/- %.2f'%
                  (np.mean(outer_scores_mse), np.std(outer_scores_mse)))
        print('        MAE %.2f +/- %.2f'%
                  (np.mean(outer_scores_mae), np.std(outer_scores_mae)))
        print('        R2 %.2f +/- %.2f'%
                  (np.mean(outer_scores_r2), np.std(outer_scores_r2)))

-------------------------------------------------- 

Algorithm: Ridge
    Inner loop:

        Best MSE (inner test folds): -5.767120933361441
        Best parameters: {'reg1__alpha': 5}
        MSE (on outer test fold) 5.7930803631196195
        MAE (on outer test fold) 2.0864026355801144
        R2 (on outer test fold) 0.22804606239525682

        Best MSE (inner test folds): -5.741263042833208
        Best parameters: {'reg1__alpha': 5}
        MSE (on outer test fold) 6.215081967714174
        MAE (on outer test fold) 2.100612834440601
        R2 (on outer test fold) -0.16965351062772305

        Best MSE (inner test folds): -4.9874697618852855
        Best parameters: {'reg1__alpha': 5}
        MSE (on outer test fold) 8.685685915940127
        MAE (on outer test fold) 2.508753071668833
        R2 (on outer test fold) 0.1495475551599761

        Best MSE (inner test folds): -5.861217935636492
        Best parameters: {'reg1__alpha': 5}
        MSE (on outer test fold) 5.9330892049

ValueError: Invalid parameter criterion for estimator Ridge(). Check the list of available parameters with `estimator.get_params().keys()`.